FUNCTION I NPV, IRR, PI

In [ ]:
import numpy as np

def calculate_irr(cashflow, time):
    guess = 0.1
    max_iter = 50
    tolerance = 1e-8
    
    for _ in range(max_iter):
        npv = np.sum(cashflow / (1 + guess) ** time)
        npv_deriv = -np.sum(time * cashflow / (1 + guess) ** (time + 1))
        
        guess = guess - npv / npv_deriv
        
        if np.abs(npv) < tolerance:
            return guess
    
    return None

def investment_assessment(cashflow, bank_rate, inflation_rate, risk, time):
    cashflow = np.array(cashflow)  # Convert cashflow list to numpy array
    bank_rate_real = (1 + bank_rate) / (1 + inflation_rate) - 1
    bank_rate_real_risk = bank_rate_real + risk
    
    npv = np.sum(cashflow / (1 + bank_rate_real_risk) ** time)
    irr = calculate_irr(cashflow, time)
    pi = npv / np.abs(cashflow[0])
    
    results = {
        "Net Present Value (NPV)": npv,
        "Internal Rate of Return (IRR)": irr,
        "Profitability Index (PI)": pi
    }
    
    return results


def get_user_input(prompt):
    while True:
        try:
            value = float(input(prompt))
            return value
        except ValueError:
            print("Invalid input! Please enter a valid number.")

def main():
    cashflow = []
    while True:
        try:
            cashflow_size = int(input("Enter the number of cash flows: "))
            for i in range(cashflow_size):
                cashflow.append(get_user_input(f"Enter cash flow for period {i + 1}: "))
            break
        except ValueError:
            print("Invalid input! Please enter an integer.")

    bank_rate = get_user_input("Enter the bank interest rate (as a decimal): ")
    inflation_rate = get_user_input("Enter the inflation rate (as a decimal): ")
    risk = get_user_input("Enter the risk premium (as a decimal): ")

    time = len(cashflow)
    bank_rate = np.full_like(time, bank_rate)
    inflation_rate = np.full_like(time, inflation_rate)
    risk = np.full_like(time, risk)

    result = investment_assessment(cashflow, bank_rate, inflation_rate, risk, time)

    print("\nResults:")
    for key, value in result.items():
        print(f"{key}: {value}")

if __name__ == "__main__":
    main()

FUNTION II installment debt repayment scheme for annuity installments

In [ ]:
def annuity_installment(debt_amount, interest_rate, time_splits, fee=0, fixed_part=False, overpay=0):

    # Adjust the debt amount for any additional fee
    if fee > 0:
        debt_amount += debt_amount * fee / 100

    installment_rate = []

    for rate_number in range(1, time_splits + 1):
        # Get the current interest rate
        current_interest_rate = interest_rate[rate_number - 1] if isinstance(interest_rate, list) and rate_number <= len(interest_rate) else interest_rate

        if fixed_part:
            # Calculate installment with a fixed capital part
            installment_capital = debt_amount / time_splits
            interest = (debt_amount - (rate_number - 1) * installment_capital) * (current_interest_rate / 100) / 12
            installment_rate.append([round(interest, 2), round(installment_capital, 2), round(interest + installment_capital, 2)])
        else:
            # Calculate standard annuity installment
            installment = round((debt_amount * (current_interest_rate / 100) / 12) / (1 - (1 + current_interest_rate / 100 / 12) ** (-time_splits)) + overpay, 2)
            installment_rate.append(installment)

    return installment_rate


def get_user_input(prompt):
    while True:
        try:
            value = float(input(prompt))
            return value
        except ValueError:
            print("Invalid input! Please enter a valid number.")


# User input
debt_amount = get_user_input("Enter the initial amount of debt: ")
time_splits = int(get_user_input("Enter the number of time splits (e.g., years): "))
fee = get_user_input("Enter an additional fee as a percentage of the initial debt amount (if any): ")
fixed_part = input("Is there a fixed part of the installment? (y/n): ").lower() == 'y'
overpay = get_user_input("Enter any additional amount overpaid with each installment: ")

# If there is a fixed part, ask for the interest rates for each time split
if fixed_part:
    interest_rate = []
    for i in range(time_splits):
        rate = get_user_input(f"Enter the interest rate for year {i + 1}: ")
        interest_rate.append(rate)
else:
    interest_rate = get_user_input("Enter the annual interest rate (as a percentage): ")

# Calculate installment scheme
installments = annuity_installment(debt_amount, interest_rate, time_splits, fee, fixed_part, overpay)

# Display installment scheme
print("\nInstallment debt repayment scheme:")
for rate_number, rate in enumerate(installments, start=1):
    print(f"Installment {rate_number}: {rate} PLN")